In [15]:
import pymysql

In [16]:
conn = pymysql.connect(
    host='mysql5049.site4now.net',
    user='aa0682_movies',
    password='Password1234.',
    db='db_aa0682_movies',
    connect_timeout=8800,
    cursorclass=pymysql.cursors.DictCursor
     
)

In [19]:
cursor = conn.cursor()
cursor.execute(''' SELECT * FROM watchedMovies''')
data = cursor.fetchall()[::-1]
conn.commit()
cursor.close()

movies = [{'movie_id':row['movie_id']} for row in data]


In [20]:
movies

[{'movie_id': 1},
 {'movie_id': 2},
 {'movie_id': 1},
 {'movie_id': 1},
 {'movie_id': 1},
 {'movie_id': 3},
 {'movie_id': 88},
 {'movie_id': 3},
 {'movie_id': 88},
 {'movie_id': 1},
 {'movie_id': 115},
 {'movie_id': 1},
 {'movie_id': 115},
 {'movie_id': 2},
 {'movie_id': 69},
 {'movie_id': 69},
 {'movie_id': 9},
 {'movie_id': 4},
 {'movie_id': 2},
 {'movie_id': 1},
 {'movie_id': 116},
 {'movie_id': 3},
 {'movie_id': 88},
 {'movie_id': 6},
 {'movie_id': 9},
 {'movie_id': 7},
 {'movie_id': 4},
 {'movie_id': 1},
 {'movie_id': 2},
 {'movie_id': 1},
 {'movie_id': 282},
 {'movie_id': 105},
 {'movie_id': 1},
 {'movie_id': 65},
 {'movie_id': 240},
 {'movie_id': 179},
 {'movie_id': 211},
 {'movie_id': 1},
 {'movie_id': 8},
 {'movie_id': 105},
 {'movie_id': 179},
 {'movie_id': 15},
 {'movie_id': 2},
 {'movie_id': 1},
 {'movie_id': 3},
 {'movie_id': 2},
 {'movie_id': 2},
 {'movie_id': 3},
 {'movie_id': 88},
 {'movie_id': 3},
 {'movie_id': 4},
 {'movie_id': 1},
 {'movie_id': 115},
 {'movie_id': 3}